In [1]:
import random
import copy

In [2]:
import numpy as np

In [3]:
import networkx as nx
import matplotlib.pyplot as plt

In [9]:
G = nx.cycle_graph(50) #initialise graph
steps = 0

#print(len(Graphs))
no_of_edges = 0
while steps < 100 : 
    #print(steps)
    r1 = random.randrange(0,50)
    d1 = G.degree[r1]

    if d1<3:
        # lower bound inclusive & upper bound not inclusive in range
        # r2 = random.sample(range(r1-5,r1-1), 4)   
        l1 = [*range(r1-5,r1-1,1)]
        # lower bound inclusive & upper bound not inclusive in range
        # r3 = random.sample(range(r1+2,r1+6), 4)
        l2 = [*range(r1+2,r1+6,1)]
        joinedList = l1 + l2

        #if the nodes go out of the range [0,49] adding/subtracting 50 to make it cyclic
        for i in range(8):
            if joinedList[i] < 0:
                joinedList[i] += 50
            elif joinedList[i] > 49:
                joinedList[i] -= 50
        edge_added = False

        stepss = 0 #using this to stop the loop when there are only few edges left
        while edge_added == False and stepss < 50:
            r2 = random.choice(joinedList)
            d2 = G.degree[r2]
            if d2<3:
                edge_added = True #the looop will keep running either if the edge has not been added or if the stepss count < 50
                G.add_edge(r1,r2)
            stepss += 1
    steps += 1

In [6]:
def init_rand_pos():
    prey = random.randint(0, 49)
    pred = random.randint(0, 49)

    agent = random.randint(0, 49)
    while agent == prey or agent == pred: #loop generates random posiotns for agent until it is different from prey/predator
        agent = random.randint(0, 49)
    return (prey, pred, agent) 

In [7]:
def move_prey(G, prey):
    move = random.randint(0, G.degree(prey)) #choosing which neighbor to move to or stay in its location(=0)
    if move == 0:
        return prey
    else:
        return [i for i in G.neighbors(prey)][move-1] #forms a list of the neighbors and chooses one of it

In [8]:
def move_pred_distract(G, pred, agent): #predator moving
    distracted = np.random.choice(2, 1, [0.6,0.4])[0] 
    #print(distracted)
    #check if distracted or not
    if distracted == 0: #not distracted, will move to the neighbor eith tthe shortest distance to agent
        min_ad = 100
        dis_pred = []
        dis_pred_choice = []

        for i in G[pred]: #finding shortest distance
            agent_dis = BFS(G, i, agent)
            dis_pred.append([i, agent_dis])
            if agent_dis < min_ad:
                min_ad = agent_dis

        for i in range(len(dis_pred)): #adding neighbors with the same shortest distance
            if min_ad == dis_pred[i][1]:
                dis_pred_choice.append(dis_pred[i][0])

        return random.choice(dis_pred_choice) #breaking ties at random
    
    else: #distacted
        return random.choice([i for i in G[pred]]) #returns random neighbor to move to


In [36]:
def BFS(G, start, finish):
    #print("SF", start, finish)
    vis = [0] * 50
    q = [[start, 0]] #adding the start node to the queue
    #path = list()
    vis[start] = 1 #mark the start node as visited
    dis = 0
    while len(q) != 0:
        node =  q.pop(0)
        #print("path" , path)
        #node = path[-1]
        if node[0] == finish :
            return node[1]
        for j in G[node[0]]:
            if vis[j] != 1:
                q.append([j, node[1] + 1]) #appending teh negihbors and the distance from the parent
                vis[j] = 1


In [849]:
U_star = np.zeros((50, 50, 50))

In [11]:
U_star[0,1,0]

0.0

In [81]:
G[49]

AtlasView({48: {}, 0: {}, 4: {}})

# states :: (Agent, Prey, Predator)

# Initialization

In [850]:
for i in range(50):
    for j in range(50):
        for k in range(50):
            for n in G[i]:
                U_star[i, j, n] = float('inf')
                U_star[i, j, i] = float('inf')  
                U_star[i, n, k] = 1
                U_star[i, i, k] = 0
            if i != j and i != k and j not in [nbor for nbor in G[i]] and k not in [nbor for nbor in G[i]]:
                U_star[i, j, k] = int(BFS(G, i, j))

In [851]:
for i in range(50):
    for j in range(50):
        for k in range(50):
            if i != j and i != k and j not in [nbor for nbor in G[i]] and k not in [nbor for nbor in G[i]]:
                min_U = float('inf')
                for n in G[i]:
                    sum_U_Pred = 0
                    sum_U_Prey = 0
                    sum_U = 0
                    #Prey Neighbor
                    p = 1/G.degree(j)
                    for m in G[j]:
                        if U_star[n, m, k] != float('inf'):
                            sum_U_Prey += p * U_star[n, m, k]
                    #Pred neighbor
                    min_nbor = 100
                    for l in G[k]:
                        nbor_dis = BFS(G, l, i)
                        if min_nbor > nbor_dis:
                            min_nbor = nbor_dis
                            nbor = l
                    p = 1/G.degree(k)    
                    for l in G[k]:
                        if U_star[n, j, l] != float('inf'):                     
                            if nbor == l: 
                                sum_U_Pred += 0.6 * U_star[n, j , l] #SHORTEST 
                            else:
                                sum_U_Pred += 0.4 * p * U_star[n, j , l] #RANDOM
                    
                    sum_U = sum_U_Prey + sum_U_Pred
                    temp_U = 1 + sum_U
                    if min_U > temp_U:
                        min_U = temp_U
                U_star[i, j, k] = min_U 

In [852]:
U_star[49,9,5]

1.8311111111111111

# Value Iteration

In [881]:
while True:
    U_star_new = np.zeros((50,50,50))
    max_S = -100
    for i in range(50):
        for j in range(50):
            for k in range(50):
                if i != j and i != k and j not in [nbor for nbor in G[i]] and k not in [nbor for nbor in G[i]]:
                    min_U = float('inf')
                    for n in G[i]:
                        sum_U_Pred = 0
                        sum_U_Prey = 0
                        sum_U = 0
                        p = 1/G.degree(j)
                        for m in G[j]:
                            if U_star[i, m, k] != float('inf'):
                                sum_U_Prey += p * U_star[n, m, k]

                        min_nbor = 100
                        for l in G[k]:
                            nbor_dis = BFS(G, l, i)
                            if min_nbor > nbor_dis:
                                min_nbor = nbor_dis
                                nbor = l
                        p = 1/(G.degree(k))
                        for l in G[k]:
                            if U_star[i, j, l] != float('inf'): 
                                if nbor == l: 
                                    sum_U_Pred += 0.6 * U_star[n, j , l]
                                else:
                                    sum_U_Pred += 0.4 * p * U_star[n, j , l]

                        sum_U = sum_U_Prey + sum_U_Pred
                        temp_U = 1 + sum_U
                        if min_U > temp_U:
                            min_U = temp_U
                    U_star_new[i, j, k] = min_U
                else:
                    U_star_new[i, j, k] = U_star[i, j, k]
                
                diff = abs(U_star_new[i, j, k] - U_star[i, j, k])
                if diff > max_S:
                    max_S = diff
                    
    U_star = np.copy(U_star_new)  
    if max_S < 0.0001:
        break
     

c:\users\asus\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:40: RuntimeWarning: invalid value encountered in double_scalars


In [856]:
max_U = -100
for i in range(50):
        for j in range(50):
            for k in range(50):
                if U_star[i, j, k] > max_U and U_star[i, j , k] != float('inf'):
                    max_U = U_star[i, j, k]
                    state = [i, j, k]

In [858]:
max_U, state

(35.36, [49, 27, 12])

# Agent using U*

In [864]:
success = 0 #agent's survival rate (when it catches the prey)
fail1 = 0 #When agent's gets caught by the predator
fail2 = 0 #when time is up
total_steps = 0
for runs in range(100):
    prey, pred, agent = init_rand_pos() #initialize random start locations
    #print("RUN #", runs)
    #print(prey, pred, agent)
    time = 0
    while time < 1000:
        next_pos = None
        min_U = float('inf')
        for i in G[agent]:
            if U_star[i, prey, pred] < min_U:
                min_U = U_star[i, prey, pred]
                next_pos = i
        

        if next_pos != None: #runs only if there any neighbor satisfies the conditions 
            agent = next_pos

        #otherwise the agent stays at its position

        #print("next_pos", next_pos)       
        if agent == prey: #break if the agent finds the prey
            break

        #it cannot move to the predator's location because if teh predator is on the agent's neighbors, the agent won't move
        #prey/predator move
        prey = move_prey(G, prey)
        if agent == prey: #break if the agent finds the prey
            break
        pred = move_pred_distract(G, pred, agent)

        #print(prey, pred, agent)
        #now if the prey/predator find agent, the run exits
        if agent == pred:
            break
        time += 1
        total_steps += 1
        #print("time",time)
    #if the prey/predator exist at the same location as the agent, then the agent dies
    if agent == pred:
        fail1 += 1
    elif agent == prey:
        success += 1
    elif time == 1000:
        fail2 += 1     
success, fail1, fail2, total_steps/100

(98, 2, 0, 5.11)


# Neural Network

In [189]:
#Base class
class Layer:
    def __init__(self):
        self.input = None
        self.output = None
    #computes the output Y of a layer for a given input X
    def forward_propagation(self, input):
        raise NotImplementedError
        
    #computes dE/dX for a given dE/dY (and update parameters if any)
    def backward_propagation(self, output_error, learning_rate):
        raise NotImplementedError

In [631]:
#inherit from base class Layer
class FCLayer(Layer):
    def __init__(self, input_size, output_size):
        self.weights = np.random.rand(input_size, output_size) - 0.5
        #self.bias = np.random.rand(1, output_size) - 0.5
        self.bias = np.zeros((1, output_size))
    #returns output for a given input
    
    def forward_propagation(self, input_data):
        self.input = input_data
        
        self.output = np.dot(self.input, self.weights) + self.bias
        #print(self.output)
        return self.output
    
    #computes dE/dW, dE/dB for a given output_error=dE/dY. Returns input_error=dE/dX.
    
    def backward_propagation(self, output_error, learning_rate):
        input_error = np.dot(output_error, self.weights.T)
        weights_error = np.dot(self.input.T, output_error)
        #dBias = output_error
        #update parameters
        self.weights -= learning_rate * weights_error
        self.bias -= learning_rate * output_error
        return input_error

In [192]:
class ActivationLayer(Layer): 
    def __init__(self, activation, activation_prime):
        self.activation = activation
        self.activation_prime = activation_prime
        
    def forward_propagation(self, input_data):
        self.input = input_data
        self.output = self.activation(self.input)
        return self.output # returns the activated input
    
    
    def backward_propagation(self, output_error, learning_rate):
        return self.activation_prime(self.input) * output_error # Returns input_error=dE/dX for a given output_error=dE/dY.

In [597]:
#activation function and its derivative
def tanh(x):
    return np.tanh(x)

def tanh_prime(x):
    return 1-np.tanh(x)**2;

In [601]:

#activation function and its derivative
def Leaky_ReLu(x):
    return np.where(x > 0, x, x * 0.01);

def ReLuDerivative(x):
    x[x<=0] = 0
    x[x>0] = 1
    return x

In [665]:
#activation function and its derivative
def Sigmoid(x):
    x = x.astype(float)
    return 1/(1 + np.exp(-x * 1.0))

def Sigmoid_Derivative(x):
    x = x.astype(float)
    f = 1/(1 + np.exp(-x * 1.0))
    return f * (1 - f)

In [194]:
#loss function and its derivative
def mse(y_true, y_pred):
    return np.mean(np.power(y_true-y_pred, 2));

def mse_prime(y_true, y_pred):
    return 2*(y_pred-y_true)/y_true.size;

In [866]:

class Network:
    def __init__(self):
        self.layers = []
        self.loss = None
        self.loss_prime = None
    #add layer to network
    def add(self, layer):
        self.layers.append(layer)
    #set loss to use
    def use(self, loss, loss_prime):
        self.loss = loss
        self.loss_prime = loss_prime
    #predict output for given input
    def predict(self, input_data):
        # sample dimension first
        samples = len(input_data)
        result = []

        #run network over all samples
        for i in range(samples):
            # forward propagation
            output = input_data[i]
            for layer in self.layers:
                output = layer.forward_propagation(output)
            result.append(output)

        return result

    #train the network
    def fit(self, x_train, y_train, epochs, learning_rate):
        # sample dimension first
        samples = len(x_train)
        #training loop
        for i in range(epochs):
            err = 0
            for j in range(samples):
                #forward propagation
                output = x_train[j]
                for layer in self.layers:
                    output = layer.forward_propagation(output)

                #compute loss (for display purpose only)
                err += self.loss(y_train[j], output)

                #backward propagation
                error = self.loss_prime(y_train[j], output)
                for layer in reversed(self.layers):
                    error = layer.backward_propagation(error, learning_rate)
            #calculate average error on all samples
            err /= samples
            print('epoch %d/%d ' % (i+1, epochs))

In [215]:
import pandas as pd

In [428]:
X_data = []
Y_data = []
for i in range(50):
    for j in range(50):
        for k in range(50):
            if i != k and k not in [nbor for nbor in G[i]]:
                X_data.append([i, j, k, G.degree(i), int(j in G[i])])
                Y_data.append(U_star[i, j, k])

In [437]:
df = pd.DataFrame(X_data, columns=['Agent', 'Prey', 'Pred', 'Degree', 'Is Prey Neighbor?'])

df.head()

,Agent,Prey,Pred,Degree,Is Prey Neighbor?
0,0,0,2,3,0
1,0,0,3,3,0
2,0,0,4,3,0
3,0,0,5,3,0
4,0,0,6,3,0


In [585]:


#graphDF.index = np.arange(0,115300)
df.index

RangeIndex(start=0, stop=115300, step=1)

In [563]:
data = np.arange(3)
graphDF = pd.DataFrame(data.reshape(1, 3))
graphDF.iloc[0] = [i for i in G[0]]
for i in range(50):
    for j in range(50):
        for k in range(50):
            if i != k and k not in [nbor for nbor in G[i]]:
                add = [i for i in G[i]]
                if len(add) == 2:
                    add.append(-1)
                graphDF.loc[len(graphDF)] = add
    print(i)
#graphDF = graphDF.iloc[1:, :]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49


In [583]:
len(graphDF)

115300

In [812]:
X_data_fin = pd.concat([df, graphDF], axis = 1)
X_data_fin.tail()

,Agent,Prey,Pred,Degree,Is Prey Neighbor?,0,1,2
115295,49,49,43,3,0,48,0,4
115296,49,49,44,3,0,48,0,4
115297,49,49,45,3,0,48,0,4
115298,49,49,46,3,0,48,0,4
115299,49,49,47,3,0,48,0,4


In [588]:
X_data_fin.isnull().values.any()

False

In [813]:
X_data_fin = np.array(X_data_fin)
Y_data =  np.array(Y_data)
idxes = np.arange(0, len(X_data_fin))
np.random.shuffle(idxes)
train_idx = int(0.8 * len(X_data_fin))
train_X_data = X_data_fin[idxes[:train_idx]]
test_X_data = X_data_fin[idxes[train_idx:]]
train_Y_data = Y_data[idxes[:train_idx]]
test_Y_data = Y_data[idxes[train_idx:]]

In [814]:
train_X_data[0], (train_Y_data[0])

(array([ 5, 17, 33,  3,  0,  4,  6,  9], dtype=int64), 1.4999996863872627)

In [815]:
train_X_data = train_X_data.reshape((92240, 1, 8))
train_Y_data = train_Y_data.reshape((92240, 1, 1))

In [867]:
net = Network()
net.add(FCLayer(8, 4))
net.add(ActivationLayer(Leaky_ReLu, ReLuDerivative))
#net.add(FCLayer(16, 8))
#net.add(ActivationLayer(Leaky_ReLu, ReLuDerivative))
#net.add(FCLayer(8, 4))
#net.add(ActivationLayer(Leaky_ReLu, ReLuDerivative))
net.add(FCLayer(4, 1))
net.add(ActivationLayer(Leaky_ReLu, ReLuDerivative))

# train
net.use(mse, mse_prime)
net.fit(train_X_data, train_Y_data, epochs=50, learning_rate=0.01)

# test
out = net.predict(test_X_data)

epoch 1/50 
epoch 2/50 
epoch 3/50 
epoch 4/50 
epoch 5/50 
epoch 6/50 
epoch 7/50 
epoch 8/50 
epoch 9/50 
epoch 10/50 
epoch 11/50 
epoch 12/50 
epoch 13/50 
epoch 14/50 
epoch 15/50 
epoch 16/50 
epoch 17/50 
epoch 18/50 
epoch 19/50 
epoch 20/50 
epoch 21/50 
epoch 22/50 
epoch 23/50 
epoch 24/50 
epoch 25/50 
epoch 26/50 
epoch 27/50 
epoch 28/50 
epoch 29/50 
epoch 30/50 
epoch 31/50 
epoch 32/50 
epoch 33/50 
epoch 34/50 
epoch 35/50 
epoch 36/50 
epoch 37/50 
epoch 38/50 
epoch 39/50 
epoch 40/50 
epoch 41/50 
epoch 42/50 
epoch 43/50 
epoch 44/50 
epoch 45/50 
epoch 46/50 
epoch 47/50 
epoch 48/50 
epoch 49/50 
epoch 50/50 


# Agent using V (Neural Network model)

In [868]:
success = 0 #agent's survival rate (when it catches the prey)
fail1 = 0 #When agent's gets caught by the predator
fail2 = 0 #when time is up
steps = 0
total_steps = 0
for runs in range(100):
    prey, pred, agent = init_rand_pos() #initialize random start locations
    #print("RUN #", runs)
    #print(prey, pred, agent)
    time = 0
    while time < 1000:
        next_pos = None
        min_U = float('inf')
        for i in G[agent]:
            #if agent != prey and agent != pred and prey not in [nbor for nbor in G[agent]] and pred not in [nbor for nbor in G[agent]]:
            inp1 = pd.DataFrame([i, prey, pred, G.degree(i), prey in G[i]]).T
            #print(inp1)
            inp2 =[z for z in G[i]]

            if len(inp2) == 2:
                inp2.append(-1)
            inp2 =  pd.DataFrame(inp2).T
            #print(inp2)
            inp = pd.concat([inp1, inp2], axis = 1)
            inp = np.array(inp)
            #print(inp)
            inp = inp.reshape((1, 1, 8))
            pred_V_val = net.predict(inp)[0][0]
            #else:
                #pred_V_val = U_star[i, prey, pred]
            #print(pred_V_val)
            if i != j and i != k and j not in [nbor for nbor in G[i]] and k not in [nbor for nbor in G[i]]:
                error = abs(U_star[i, j, k] - pred_V_val)
                steps += 1
            if pred_V_val < min_U:
                min_U = pred_V_val
                next_pos = i
        

        if next_pos != None: #runs only if there any neighbor satisfies the conditions 
            agent = next_pos

        #otherwise the agent stays at its position

        #print("next_pos", next_pos)       
        if agent == prey: #break if the agent finds the prey
            break

        #it cannot move to the predator's location because if teh predator is on the agent's neighbors, the agent won't move
        #prey/predator move
        prey = move_prey(G, prey)
        if agent == prey: #break if the agent finds the prey
            break
        pred = move_pred_distract(G, pred, agent)

        #print(prey, pred, agent)
        #now if the prey/predator find agent, the run exits
        if agent == pred:
            break
        time += 1
        total_steps += 1
        #print("time",time)
    #if the prey/predator exist at the same location as the agent, then the agent dies
    if agent == pred:
        fail1 += 1
    elif agent == prey:
        success += 1
    elif time == 1000:
        fail2 += 1 
success, fail1, fail2, total_steps/100

(94, 6, 0, 5.89)


In [822]:
error[0]/steps

0.004519954262657276

In [647]:
np.random.rand(1, 4)

array([[0.42307161, 0.74331679, 0.60536248, 0.49241947]])

In [630]:
np.zeros((1,4))

array([[0., 0., 0., 0.]])

In [658]:
np.exp(-0)

1.0

# Partial Prey Location

In [678]:
'''initialized with 1/49 probability for all the nodes in the graph where the agent is not
located and 0 for the node where the agent is present since the prey/predator cannot start at those locations.'''
def init_prey_belief(agent):
    belief = [1/49] * 50 
    belief[agent] = 0
    return belief

In [679]:
#When the survey drone finds the prey/predator, the agent updates the belief with 1 at the node where the prey/predator exists and 0 for all other nodes.
def found(survey_node):
    belief = [0] * 50
    belief[survey_node] = 1
    return belief

In [680]:
def not_found(belief, G, survey_node):
    
    not_in_survey_prob = 1 - belief[survey_node]
    belief[survey_node] = 0
    for i in range(50):
        belief[i] = belief[i]/not_in_survey_prob

    return belief

In [681]:
def move_prey_belief(belief, G):
    belief_new = [0] * 50
    for i in range(50):
        for j in G[i]:
            belief_new[i] += belief[j] * (1/(G.degree(j)+1)) #adds the probability for the prey to move from neighbor j -> i
        belief_new[i] += belief[i] * (1/(G.degree(i)+1)) #adds the probability of prey staying at its location
        
    return belief_new

In [682]:
def max_prey_belief(belief):
    max_value = max(belief) #finds maximum value
    choice_ =  []
    for i in range(50):
        if belief[i] == max_value: #finds all nodes with same highest probability
            choice_.append(i)
    return random.choice(choice_) #breaking ties at random

In [710]:
#ceratin if there is only one node with 1 probability in the graph and rest are 0
def certain(belief):
    count = 0
    for i in range(50):
        if belief[i] == 1:
            count += 1
    return count == 1 

In [683]:
U_partial = np.copy(U_star)

In [825]:
U_partial_states = np.zeros((50, 50, 50))
U_partial_occ = []

# Agent Using U_partial

In [869]:

success = 0
fail1 = 0
fail2 = 0
total_steps = 0
for runs in range(1000):
    prey, pred, agent = init_rand_pos() #initialize random start locations
    #print("RUN #", runs)
    #print("init", prey, pred, agent)
    belief = init_prey_belief(agent) #initialize belief for prey (Prior Belief)
    time = 0
    while time < 1000:
        #print(belief)
        if certain(belief) != True: #only survey if agent is not certain about prey's location
            survey_node = max_prey_belief(belief)
            #print("survey_node", survey_node)
            #print("before", belief, sum(belief))
            if prey == survey_node:
                belief = found(survey_node) #update belief (observational prob)
            else:
                belief = not_found(belief, G, survey_node) #update belief (observational prob)
                
        sum_U = 0
        for i in range(50):
            sum_U += belief[i] * U_star[agent, i, pred]
        U_partial[agent, i, pred] = sum_U
 
        U_partial_occ.append([agent, pred, prey])
        for i in range(50):
            U_partial_states[agent, pred, i] = belief[i]
        
        #print("after survey", belief, sum(belief))

        prey_belief_loc = max_prey_belief(belief) #prey's location at highest node belief
        #print("prey_belief", prey_belief)
        
        
        
        min_U = float('inf')
        for i in G[agent]:
            if U_partial[i, prey_belief_loc, pred] < min_U:
                min_U = U_partial[i, prey_belief_loc, pred]
                next_node = i
         
        agent = next_node

        #print("next_pos", next_pos)       
        if agent == prey: #break if the agent finds the prey
            break

        belief = not_found(belief, G, agent) #update belief (observational prob)

         #prey/predator move
        prey = move_prey(G, prey)

        if agent == prey:
            break

        belief = move_prey_belief(belief, G) #update belief (transitional prob)

        #print("trans", belief, sum(belief))
        pred = move_pred_distract(G, pred, agent)
        #print("move", prey, pred, agent)
         #now if the predator find agent, the run exits
        if agent == pred:
            break

        time += 1
        #print("time",time)
        total_steps += 1
    #if the prey/predator exist at the same location as the agent, then the agent dies
    if agent == pred:
        fail1 += 1
    elif agent == prey:
        success += 1
    elif time == 1000:
        fail2 += 1

success/10, fail1/10, fail2/10, total_steps/1000

c:\users\asus\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in double_scalars


(87.3, 12.7, 0, 6.32)


# V_partial

In [836]:
X_data_p = []
Y_data_p = []
for i in range(len(U_partial_occ)):
    
    agent = U_partial_occ[i][0]
    pred = U_partial_occ[i][1]
    prey = U_partial_occ[i][2]
    if(U_partial[agent, prey, pred]) != float('inf'):
        X_data_p.append([agent, pred, G.degree(agent), G.degree(pred)])

        nbor_agent = [i for i in G[agent]]
        nbor_pred = [i for i in G[pred]] 
        for i in nbor_agent:
            X_data_p[-1].append(i)
        for i in nbor_pred:
            X_data_p[-1].append(i)

        belief = U_partial_states[agent, pred]
        for i in range(50):
            X_data_p[-1].append(belief[i])


        Y_data_p.append([U_partial[agent, prey, pred]])

X_data_p = pd.DataFrame(X_data_p)
Y_data_p = pd.DataFrame(Y_data_p)
X_data_p, Y_data_p

(       0   1   2   3   4   5   6   7     8          9     ...           50  \
 0      18   1   3   3  17  19  22   0   2.0   3.000000    ...     0.021935   
 1      17   2   3   3  16  18  19   1   3.0   7.000000    ...     0.020464   
 2      18   3   3   3  17  19  22   2   4.0   1.000000    ...     0.019389   
 3      17   1   3   3  16  18  19   0   2.0   3.000000    ...     0.024648   
 4      16   0   2   3  15  17   1  49  45.0   0.021779    ...     0.016334   
 5      17   1   3   3  16  18  19   0   2.0   3.000000    ...     0.024648   
 6      18   2   3   3  17  19  22   1   3.0   7.000000    ...     0.023634   
 7      17   7   3   3  16  18  19   6   8.0   2.000000    ...     0.027129   
 8      16   6   2   3  15  17   5   7   8.0   0.022595    ...     0.023066   
 9      15   5   3   3  14  16  10   4   6.0   9.000000    ...     0.024994   
 10     14   9   3   3  13  15  12   8  10.0   5.000000    ...     0.025630   
 11     38  10   2   3  37  39   9  11  15.0   0.020

In [837]:
data_nan = pd.concat((X_data_p, Y_data_p), axis = 1)
data_nan

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,0
0,18,1,3,3,17,19,22,0,2.0,3.000000,...,0.020877,0.021430,0.021682,0.022115,0.023172,0.022700,0.016941,0.023120,0.024671,6.833333
1,17,2,3,3,16,18,19,1,3.0,7.000000,...,0.019494,0.020278,0.021041,0.021527,0.022127,0.021568,0.016099,0.022057,0.023743,10.400000
2,18,3,3,3,17,19,22,2,4.0,1.000000,...,0.016985,0.024714,0.018470,0.024672,0.024813,0.000000,0.019429,0.025632,0.024592,4.711111
3,17,1,3,3,16,18,19,0,2.0,3.000000,...,0.024382,0.023858,0.024064,0.023775,0.024230,0.025273,0.019291,0.025571,0.025135,3.888889
4,16,0,2,3,15,17,1,49,45.0,0.021779,...,0.021779,0.021779,0.021779,0.021779,0.023593,0.018149,0.023593,0.021779,NaN,3.977778
5,17,1,3,3,16,18,19,0,2.0,3.000000,...,0.024382,0.023858,0.024064,0.023775,0.024230,0.025273,0.019291,0.025571,0.025135,4.555556
6,18,2,3,3,17,19,22,1,3.0,7.000000,...,0.022956,0.022574,0.020881,0.015787,0.018583,0.019967,0.019967,0.000000,0.025390,11.266667
7,17,7,3,3,16,18,19,6,8.0,2.000000,...,0.027345,0.027182,0.027106,0.026626,0.024997,0.024862,0.018567,0.024332,0.022612,4.623704
8,16,6,2,3,15,17,5,7,8.0,0.022595,...,0.022595,0.023066,0.022595,0.023066,0.018045,0.012396,0.018045,0.016946,NaN,5.800000
9,15,5,3,3,14,16,10,4,6.0,9.000000,...,0.025365,0.023494,0.023368,0.022043,0.019765,0.018123,0.013206,0.017682,0.017757,12.600000


In [871]:
X_data_p.isnull().values.any()

False

In [844]:
data_nan = data_nan.dropna()
data_nan.isnull().values.any()

False

In [872]:
X_data_p = data_nan.iloc[:, :-1]
Y_data_p = data_nan.iloc[:, -1]
X_data_p, Y_data_p

(       0   1   2   3   4   5   6   7     8     9     ...           50  \
 0      18   1   3   3  17  19  22   0   2.0   3.0    ...     0.021935   
 1      17   2   3   3  16  18  19   1   3.0   7.0    ...     0.020464   
 2      18   3   3   3  17  19  22   2   4.0   1.0    ...     0.019389   
 3      17   1   3   3  16  18  19   0   2.0   3.0    ...     0.024648   
 5      17   1   3   3  16  18  19   0   2.0   3.0    ...     0.024648   
 6      18   2   3   3  17  19  22   1   3.0   7.0    ...     0.023634   
 7      17   7   3   3  16  18  19   6   8.0   2.0    ...     0.027129   
 9      15   5   3   3  14  16  10   4   6.0   9.0    ...     0.024994   
 10     14   9   3   3  13  15  12   8  10.0   5.0    ...     0.025630   
 14     37   9   3   3  36  38  34   8  10.0   5.0    ...     0.020833   
 15     34  10   3   3  33  35  37   9  11.0  15.0    ...     0.016842   
 16     35   9   3   3  34  36  31   8  10.0   5.0    ...     0.024747   
 17     34   5   3   3  33  35  37   4

In [873]:
X_data_p_fin = np.array(X_data_p)
Y_data_p_fin =  np.array(Y_data_p)
idxes = np.arange(0, len(X_data_p_fin))
np.random.shuffle(idxes)
train_p_idx = int(0.8 * len(X_data_p_fin))
train_X_data_p = X_data_p_fin[idxes[:train_p_idx]]
test_X_data_p = X_data_p_fin[idxes[train_p_idx:]]
train_Y_data_p = Y_data_p_fin[idxes[:train_p_idx]]
test_Y_data_p = Y_data_p_fin[idxes[train_p_idx:]]

In [874]:
len(train_X_data_p)

11451

In [875]:
train_X_data_p = train_X_data_p.reshape((11451, 1, 60))
train_Y_data_p = train_Y_data_p.reshape((11451, 1, 1))

In [876]:
net_P = Network()
net_P.add(FCLayer(60, 10))
net_P.add(ActivationLayer(Leaky_ReLu, ReLuDerivative))
net_P.add(FCLayer(10, 1))
net_P.add(ActivationLayer(Leaky_ReLu, ReLuDerivative))

# train
net_P.use(mse, mse_prime)
net_P.fit(train_X_data_p, train_Y_data_p, epochs=50, learning_rate=0.01)


epoch 1/50 
epoch 2/50 
epoch 3/50 
epoch 4/50 
epoch 5/50 
epoch 6/50 
epoch 7/50 
epoch 8/50 
epoch 9/50 
epoch 10/50 
epoch 11/50 
epoch 12/50 
epoch 13/50 
epoch 14/50 
epoch 15/50 
epoch 16/50 
epoch 17/50 
epoch 18/50 
epoch 19/50 
epoch 20/50 
epoch 21/50 
epoch 22/50 
epoch 23/50 
epoch 24/50 
epoch 25/50 
epoch 26/50 
epoch 27/50 
epoch 28/50 
epoch 29/50 
epoch 30/50 
epoch 31/50 
epoch 32/50 
epoch 33/50 
epoch 34/50 
epoch 35/50 
epoch 36/50 
epoch 37/50 
epoch 38/50 
epoch 39/50 
epoch 40/50 
epoch 41/50 
epoch 42/50 
epoch 43/50 
epoch 44/50 
epoch 45/50 
epoch 46/50 
epoch 47/50 
epoch 48/50 
epoch 49/50 
epoch 50/50 


# Agent using V_partial

In [878]:
success = 0
fail1 = 0
fail2 = 0
steps = 0
total_steps = 0
for runs in range(100):
    prey, pred, agent = init_rand_pos() #initialize random start locations
    #print("RUN #", runs)
    #print("init", prey, pred, agent)
    belief = init_prey_belief(agent) #initialize belief for prey (Prior Belief)
    time = 0
    while time < 1000:
        #print(belief)
        if certain(belief) != True: #only survey if agent is not certain about prey's location
            survey_node = max_prey_belief(belief)
            #print("survey_node", survey_node)
            #print("before", belief, sum(belief))
            if prey == survey_node:
                belief = found(survey_node) #update belief (observational prob)
            else:
                belief = not_found(belief, G, survey_node) #update belief (observational prob)
            
            
        #print("after survey", belief, sum(belief))

        prey_belief_loc = max_prey_belief(belief) #prey's location at highest node belief
        #print("prey_belief", prey_belief)
        
        
       
        min_U = float('inf')
        for i in G[agent]:
            inp1 = []
            inp1.append([i, pred, G.degree(i), G.degree(pred)])

            nbor_i = [i for i in G[agent]]
            if len(nbor_i) == 2:
                nbor_i.append(-1)
            nbor_pred = [i for i in G[pred]] 
            if len(nbor_pred) == 2:
                nbor_pred.append(-1)
            for i in nbor_i:
                inp1[-1].append(i)
            for i in nbor_pred:
                inp1[-1].append(i)

            for i in range(50):
                inp1[-1].append(belief[i])
            
            inp = np.array(pd.DataFrame(inp1).T)
            inp = inp.reshape((1, 1, 60))
            out = net_P.predict(inp)[0]
            if i != prey and i != pred and prey not in [nbor for nbor in G[i]] and pred not in [nbor for nbor in G[i]]:
                error = abs(U_partial[i, j, k] - out)
                steps += 1
            if out < min_U:
                min_U = out
                next_node = i
         
        agent = next_node

        #print("next_pos", next_pos)       
        if agent == prey: #break if the agent finds the prey
            break

        belief = not_found(belief, G, agent) #update belief (observational prob)

         #prey/predator move
        prey = move_prey(G, prey)

        if agent == prey:
            break

        belief = move_prey_belief(belief, G) #update belief (transitional prob)

        #print("trans", belief, sum(belief))
        pred = move_pred_distract(G, pred, agent)
        #print("move", prey, pred, agent)
         #now if the predator find agent, the run exits
        if agent == pred:
            break

        time += 1
        #print("time",time)
        total_steps += 1

    #if the prey/predator exist at the same location as the agent, then the agent dies
    if agent == pred:
        fail1 += 1
    elif agent == prey:
        success += 1
    elif time == 1000:
        fail2 += 1

success/10, fail1/10, fail2/10, error/steps, total_steps/1000

(82, 18, 0, 0.0052, 6.26)
